# Notebook for generating prompts for code generation using local data
* Use most recently looked at files

In [6]:
import os

from jupyter_notebook_parser import JupyterNotebookParser

In [2]:
project_dir = "/Users/connorparish/code/hindsight_parsing/"

In [8]:
files_to_use = ['notebooks/annotation_export.ipynb', 'annotation_db.py']

In [9]:
f = files_to_use[0]
f_path = os.path.join(project_dir, f)

In [18]:
def parse_notebook(path):
    parsed = JupyterNotebookParser(f_path)
    cells = parsed.get_code_cells()
    code = ""
    for cell in cells:
        code += "".join(cell['source']) + "\n"
    return code

In [24]:
code_context = ""
for f in files_to_use:
    f_path = os.path.join(project_dir, f)
    code_context += f"Code from {f}" + "\n"
    if f.split('.')[-1] == "ipynb":
        code_context += parse_notebook(f_path)
    else:
        with open(f_path, 'r') as infile:
            code_context += infile.read()

In [31]:
task = "Create a script annotate.py that runs uses a yolo model to create annotations for a list of images and inserts them into the database."

In [34]:
prompt = "Project Context:\n" + code_context + "\n\nUsing the above project context perform the following task. " + task

In [35]:
print(prompt)

Project Context:
Code from notebooks/annotation_export.ipynb
import os
import json
import shutil

from ultralytics import YOLO
from PIL import Image

import sys
sys.path.insert(0, "../../hindsight/hindsight_server/")

from db import HindsightDB
from utils import make_dir
db = HindsightDB()
frames = db.get_frames(impute_applications=False)
frames = frames.loc[frames['application'] == "Twitter"]
sel_frames = frames.sample(n=100)
dest_dir = "../data/annotations/twitter/rand_upload_2/"
make_dir(dest_dir)
for f in sel_frames['path']:
    dest_f = os.path.join(dest_dir, os.path.basename(f))
    shutil.copy(f, dest_dir)
trained_model = YOLO("./runs/detect/train15/weights/best.pt")
pred_images_dir = dest_dir 
images = list()
for f in os.listdir(pred_images_dir):
    im = Image.open(os.path.join(pred_images_dir, f))
    images.append(im)
results = trained_model(images)
all_preds = list()
for result in results:
    image_preds_d = {}
    image_path = f"/data/local-files/?d={result.path}"
    ima